In [1]:
#!pip install --upgrade transformers==4.37.2

In [2]:
# Don't even know if these are necessary, but most models have required them
#!pip install einops accelerate

In [3]:
#!pip install flash-attn --no-build-isolation

In [4]:
#!pip install timm

In [5]:
import math
import torch
from transformers import AutoTokenizer, AutoModel

In [6]:
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    num_layers = {
        'InternVL2-1B': 24, 'InternVL2-2B': 24, 'InternVL2-4B': 32, 'InternVL2-8B': 32,
        'InternVL2-26B': 48, 'InternVL2-40B': 60, 'InternVL2-Llama3-76B': 80}[model_name]
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

In [7]:
path = "OpenGVLab/InternVL2-Llama3-76B"
device_map = split_model('InternVL2-Llama3-76B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-Llama3-76B:
- modeling_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resum

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

model-00001-of-00032.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00032.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00032.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

model-00030-of-00032.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00031-of-00032.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00032-of-00032.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [8]:
import numpy as np
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode

In [9]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

In [11]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

In [12]:
def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

In [13]:
import requests
from io import BytesIO

def load_image(image_file, input_size=448, max_num=12):
    # Check if the input is a URL
    if isinstance(image_file, str) and image_file.startswith("http"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(img) for img in images]
    pixel_values = torch.stack(pixel_values)
    
    return pixel_values

In [14]:
urls = [
 'https://samplebook.photos/img/4786a.jpg',
 'https://samplebook.photos/img/284v.jpg',
 'https://samplebook.photos/img/3193c.jpg',
 'https://samplebook.photos/img/948d.jpg',
 'https://samplebook.photos/img/3191i.jpg',
 'https://samplebook.photos/img/4788oo.jpg',
 'https://samplebook.photos/img/2086d.jpg',
 'https://samplebook.photos/img/4781r.jpg',
 'https://samplebook.photos/img/5521n.jpg',
 'https://samplebook.photos/img/2073cc.jpg',
 'https://samplebook.photos/img/2085b.jpg',
 'https://samplebook.photos/img/4782c.jpg',
 'https://samplebook.photos/img/4783v.jpg',
 'https://samplebook.photos/img/4912gg.jpg',
 'https://samplebook.photos/img/304h.jpg',
 'https://samplebook.photos/img/2078i.jpg',
 'https://samplebook.photos/img/2077q.jpg',
 'https://samplebook.photos/img/3192h.jpg',
 'https://samplebook.photos/img/904x.jpg',
 'https://samplebook.photos/img/4792b.jpg',
 'https://samplebook.photos/img/5301p.jpg',
 'https://samplebook.photos/img/5207j.jpg',
 'https://samplebook.photos/img/1007p.jpg',
 'https://samplebook.photos/img/17l.jpg',
 'https://samplebook.photos/img/286t.jpg',
 'https://samplebook.photos/img/287g.jpg',
 'https://samplebook.photos/img/3189j.jpg',
 'https://samplebook.photos/img/4911d.jpg',
 'https://samplebook.photos/img/4785a.jpg',
 'https://samplebook.photos/img/4793a.jpg'
]

In [15]:
prompt = 'There are two objects in this image. One is a color checker, and the other is a photographic sample book, open to a page containing a photograph. Give a detailed description of the photograph.'

In [16]:
def get_response(url):
    # set the max number of tiles in `max_num`
    pixel_values = load_image(url, max_num=12).to(torch.bfloat16).cuda()
    generation_config = dict(max_new_tokens=1024, do_sample=True)

    question = f'<image>\n{prompt}'
    response = model.chat(tokenizer, pixel_values, question, generation_config)

    return response

In [17]:
responses_intern = {}

for url in urls:
    response = get_response(url)
    print(url,"\n",response,"\n\n")
    responses_intern[url] = {'OpenGVLab/InternVL2-Llama3-76B':response}

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


https://samplebook.photos/img/4786a.jpg 
 The photograph presented in the open sample book depicts a rugged, rocky terrain that appears harsh and inhospitable. The image is black and white, enhancing the contrast between the shadows and the brightly illuminated areas. On the right side of the picture, a steep, uneven incline is visible, showcasing layered rock formations with sharp, jagged edges. The textures of the rocks are highly pronounced, revealing intricate details of their surfaces. Small, indistinct figures, likely humans, can be seen climbing or engaged in some activity on the rock face, though their specific actions are difficult to discern due to the distance. The scene conveys a sense of adventure and the challenges of navigating such terrain, suggesting a possible historical or exploratory context. The overall impression is one of strength and resilience, both in the natural landscape and, by implication, in the human figures interacting with it. 




Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


https://samplebook.photos/img/284v.jpg 
 The photograph in the sample book is a black-and-white image depicting a high-voltage electrical substation. The focus is on two tall, cylindrical insulators that stand vertically on the substation floor. Each insulator has a top part featuring a cluster of bushings and lines that ascend and connect to the overhead lattice of power lines and structures. The metallic connectors at the top of the insulators suggest that they are part of the transmission network, facilitating the flow of electricity. In the background, a dense network of metal trusses and support structures is visible, forming a complex web that typifies the architecture of electrical substations.

The word "VARILOUR" is printed at the bottom left of the page, followed by the statement "WHITE GLOSSY." Additionally, there is a small logo of "KOPION" in the bottom center area of the page. This information likely indicates that the photograph is an example of how images appear when pr

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


...

https://samplebook.photos/img/4911d.jpg 
 The photograph in the sample book captures a pastoral scene set against a large, leafy tree which dominates the backdrop. The scene depicts a rustic environment, likely from a time in history characterized by simpler, rural lifestyles.

In the foreground, two bovines are central to the image -- a cow and an ox, each exhibiting distinctive markings. The cow, positioned on the left side of the image, stands slightly behind the ox with its head turned towards the ox, revealing a gentle demeanor. The ox, on the right side, is in profile and appears to be calmly poised in front of the cow, with its large horns directed forwards. Both animals are depicted in a natural setting, which enhances the tranquil ambiance of the scene.

Adjacent to the animals, on the left side, there is a small structure that resembles a thatched hut. The roof of the hut is indicative of traditional construction methods commonly used in rural or village settings. A few slend

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


https://samplebook.photos/img/4785a.jpg 
 The image showcases a photographic sample book opened to a page containing a photograph. Let's delve into the intricate details of the photograph:

1. **Subject**:
   - The photograph features a figure lying down in a relaxed, perhaps dreamlike or serene pose. This figure occupies a significant portion of the image and is the central focus. 
   - The face of the individual is visible, with delicate facial features and a calm expression, adding to the tranquil mood of the image.

2. **Color and Tone**:
   - The photograph predominantly employs muted, cool tones, including shades of blues, whites, and soft grays. These colors contribute to the ethereal and peaceful atmosphere of the image.
   - The use of light and shadow enhances the visual depth, giving the photograph a slightly surreal quality. There is a delicate interplay between light areas and darker regions, emphasizing the contours and textures within the photograph.

3. **Composition**:

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


https://samplebook.photos/img/4793a.jpg 
 The photograph in the sample book depicts a person engaged in a domestic activity. Here are the detailed observations:

1. **Subject of the Photograph**: The main subject of the photograph is a seated individual, presumably a woman given the visible attire and hairstyle. She appears to be involved in laundry work.

2. **Activity Depicted**: The individual is positioned over what looks like a washbasin or a large bucket, indicating that she is washing clothes. Her arms and upper body are involved in the movement, suggesting she might be wringing out or scrubbing the garments. The action captured seems vigorous, as evidenced by the water splashes around.

3. **Attire**: The person is dressed in a light-colored, possibly striped garment. The attire looks practical, suitable for physical work and possibly made from a material that can endure getting wet.

4. **Props and Environment**: The subject is using some form of a washboard or a scrubbing imp

In [18]:
import pickle

In [19]:
with open("responses_intern.pkl", "wb") as f:
    pickle.dump(responses_intern, f)